# Import

In [1]:
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image
import cv2
from tqdm.auto import tqdm
import os
import json
import gc
import warnings
import random
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from pytorchvideo.models.hub.slowfast import slowfast_16x8_r101_50_50

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

# Seed

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False 

seed_everything(42)

# Config

In [3]:
device = torch.device("cuda:0")

flags={}
flags['batch_size'] = 2
flags['num_worker'] = 0
flags['learning_rate'] = 5e-5
flags['epoch'] = 100
flags['size'] = 256
flags['mean'] = [0.45, 0.45, 0.45]
flags['std'] = [0.225, 0.225, 0.225]
flags['num_frames'] = 64
flags['sampling_rate'] = 1
flags['slowfast_alpha'] = 4
flags['save_folder'] = './model_weights'
flags['warmup_ratio'] = 0.1

# Dataset

In [4]:
class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // flags['slowfast_alpha']
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

In [5]:
paths = sorted(glob('/data/competition/Another/Busan/Competition_1/행위/행위/*/*'))

folder_path = []
for i in paths:
    if (i[-4:] not in "json") and (i[-3:] not in "mp4"):
        folder_path.append(i)

for j in folder_path:
    try:
        os.rmdir(j)
    except:
        continue

print("빈 폴더 제거 전 :", len(folder_path))

paths = sorted(glob('/data/competition/Another/Busan/Competition_1/행위/행위/*/*'))

folder_path = []
for i in paths:
    if (i[-4:] not in "json") and (i[-3:] not in "mp4"):
        folder_path.append(i)
        
print("빈 폴더 제거 후 :", len(folder_path))

df_train = pd.DataFrame()
df_val = pd.DataFrame()


train_paths = []
val_paths = []
for num, i in enumerate(range(len(folder_path))):
    if num % 5 == 0:
        val_paths.append(folder_path[i])
    else:
        train_paths.append(folder_path[i])
        
print('train개수:', len(train_paths),' | ',  'val개수:', len(val_paths))
df_train['file_path'] = train_paths
df_val['file_path'] = val_paths

빈 폴더 제거 전 : 5088
빈 폴더 제거 후 : 5088
train개수: 4070  |  val개수: 1018


In [6]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, 
               df, 
               mode='train', 
               size=flags['size'], 
               num_frames=flags['num_frames'], 
               sampling_rate=flags['sampling_rate']):
    self.df=df
    self.mode=mode
    self.size=size
    self.num_frames=num_frames
    self.sampling_rate=sampling_rate
    self.slowfast_preproc=PackPathway()

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    file_path=self.df['file_path'].iloc[idx]
    img_path=sorted(glob(file_path + '/*.png'))
    if self.mode=='train':
        start=random.choice(np.arange(len(img_path)-self.sampling_rate*self.num_frames-1))
    else:
        start=0
    imgs=torch.zeros(self.num_frames, 3, self.size, self.size)
    for i, x in enumerate(range(start, start+self.sampling_rate*self.num_frames, self.sampling_rate)):
        img = np.array(Image.open(img_path[x]))
        imgs[i]=self.transform_func()(image=img)['image']
    
    imgs = self._pad(imgs)
    frames=self.slowfast_preproc(imgs.permute(1, 0, 2, 3))
    label = 0
    if self.mode=='train':
        if "smoking" in file_path:
            label = 0
        elif "fishing" in file_path:
            label = 1
        elif "trash_dump" in file_path:
            label = 2
        elif "wall_over" in file_path:
            label = 3
        elif "damage_to_facilities" in file_path:
            label = 4
        elif "banner_action" in file_path:
            label = 5
        elif "fliers_action" in file_path:
            label = 5
        elif "tent_setup" in file_path:
            label = 6
    else:
        label=0
    return frames, label

  def transform_func(self):
    return A.Compose([
                      A.Resize(flags['size'],flags['size']), 
                      A.Normalize(mean=flags['mean'], std=flags['std']),
                      ToTensorV2(p=1.0)
                      ])
    
  def _pad(self, imgs):
      if imgs.shape[0] < self.num_frames:
          T, C, H, W = imgs.shape
          pad = torch.zeros(self.num_frames-T, C, H, W)
          imgs = torch.cat([imgs, pad], dim=0)
      else:
          imgs = imgs[:self.num_frames]

      return imgs

In [7]:
train_dataset = CustomDataset(df=df_train)
val_dataset = CustomDataset(df=df_val)

train_dataloader = DataLoader(train_dataset, batch_size=flags['batch_size'], num_workers=flags['num_worker'], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=flags['batch_size'], num_workers=flags['num_worker'], shuffle=False)

# Model

In [8]:
class CustomModel(nn.Module):
  def __init__(self, device):
    super().__init__()
    self.model=slowfast_16x8_r101_50_50(pretrained=True)
    self.model.blocks[6].proj = nn.Linear(self.model.blocks[6].proj.in_features, 7, bias=True)
    self.to(device)
        
  def forward(self, x):  
    x = self.model(x)
    return x

In [9]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [10]:
custom_model=CustomModel(device=device)

In [11]:
sample_batch=next(iter(train_dataloader))
with torch.no_grad():
    data = [x.to(device) for x in sample_batch[0]]
    out=custom_model(data)

In [12]:
sample_batch[1]

tensor([6, 6])

In [13]:
out

tensor([[ 0.0283, -0.1038, -0.0776, -0.0389, -0.0494, -0.1633, -0.0170],
        [-0.0943,  0.0259, -0.0614, -0.1077, -0.0682, -0.4256, -0.0489]],
       device='cuda:0')

# Train

In [14]:
class CustomTrainer:
  def __init__(self, model, folder):
    self.model=model
    
    self.save_dir = f'./{folder}'
    if not os.path.exists(self.save_dir):
      os.makedirs(self.save_dir)

    param_optimizer = list(self.model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(
            nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(
            nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    self.optimizer = AdamW(optimizer_grouped_parameters, lr=flags['learning_rate'], correct_bias=False)#torch.optim.AdamW(optimizer_grouped_parameters, lr=flags['learning_rate'])

    total_steps = int(len(train_dataset)*flags['epoch']/(4*(flags['batch_size'])))
    warmup_steps = int(total_steps * flags['warmup_ratio'])
    print('total_steps: ', total_steps)
    print('warmup_steps: ', warmup_steps)
    self.scheduler = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    
    #weights = torch.FloatTensor([0.16, 0.16, 0.20, 0.16, 0.16, 0.16]).to(device)
    self.loss_fn = nn.CrossEntropyLoss()#weight=weights)

    self.best_acc = 0.0

  def run(self, train_loader, val_loader):
    print(flags)
    for epoch in range(9, flags['epoch']):
      gc.collect()
      print('learning_rate: ', self.optimizer.param_groups[0]['lr'])
      
      print(f'----- train, epoch{epoch+1} -----')
      train_loss = self.train_function(train_loader)
      print(f'train_loss: {train_loss:.9f}')

      print('----------------------------------')

      print(f'----- val, epoch{epoch+1} -----')
      with torch.no_grad():
        val_loss, val_acc = self.val_function(val_loader)
      print(f'val_loss: {val_loss:.9f}, val_acc: {val_acc:.9f}')

      torch.save(self.model.state_dict(),self.save_dir+f"/best-acc_{epoch}.bin")
      self.best_acc=val_acc
      print(f'model is saved when epoch is : {epoch+1}')

      print('----------------------------------')
      print(' ')
      if epoch == 20:break

      
  def train_function(self, train_loader):
    self.model.train()

    total_loss = 0.0
    for bi, data in tqdm(enumerate(train_loader)):
      frames, label = data
      frames = [x.to(device) for x in frames]
      label = label.to(device)

      self.optimizer.zero_grad()
      out = self.model(frames)
      loss = self.loss_fn(out.reshape(-1, 7), label.reshape(-1))
      loss.backward()
      self.optimizer.step()
      
      self.scheduler.step()
      total_loss+=loss.detach().cpu()

    return total_loss/len(train_loader)

  def val_function(self, val_loader):
    self.model.eval()

    total_loss = 0.0
    preds, targets = [], []
    for bi, data in tqdm(enumerate(val_loader)):
      frames, label = data
      frames = [x.to(device) for x in frames]
      label = label.to(device)

      out = self.model(frames)
      loss = self.loss_fn(out.reshape(-1, 7), label.reshape(-1))

      total_loss+=loss.detach().cpu()

      pred = out.argmax(1).detach().cpu().tolist()
      target = label.reshape(-1).detach().cpu().tolist()

      preds.extend(pred)
      targets.extend(target)
    
    acc = accuracy_score(targets, preds)
    return total_loss/len(val_loader), acc

In [15]:
# ensemble1
seed_everything(42)
custom_model=CustomModel(device=device)
checkpoint = torch.load('model1_weight/best-acc_8.bin', map_location=device)
custom_model.load_state_dict(checkpoint)

train_dataloader = DataLoader(train_dataset, batch_size=flags['batch_size'], num_workers=flags['num_worker'], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=flags['batch_size'], num_workers=flags['num_worker'], shuffle=False)

custom_trainer = CustomTrainer(model=custom_model, folder='model1_weight')
custom_trainer.run(train_dataloader, val_dataloader)

total_steps:  50875
warmup_steps:  5087
{'batch_size': 2, 'num_worker': 0, 'learning_rate': 5e-05, 'epoch': 100, 'size': 256, 'mean': [0.45, 0.45, 0.45], 'std': [0.225, 0.225, 0.225], 'num_frames': 64, 'sampling_rate': 1, 'slowfast_alpha': 4, 'save_folder': './model_weights', 'warmup_ratio': 0.1}
learning_rate:  0.0
----- train, epoch10 -----



train_loss: 0.009718696
----------------------------------
----- val, epoch10 -----



val_loss: 0.040678281, val_acc: 0.990176817
model is saved when epoch is : 10
----------------------------------
 
learning_rate:  2.0001965795164146e-05
----- train, epoch11 -----



train_loss: 0.028430451
----------------------------------
----- val, epoch11 -----



val_loss: 0.209480152, val_acc: 0.974459725
model is saved when epoch is : 11
----------------------------------
 
learning_rate:  4.000393159032829e-05
----- train, epoch12 -----


...


val_loss: 0.145602167, val_acc: 0.982318271
model is saved when epoch is : 19
----------------------------------
 
learning_rate:  3.749950483441648e-05
----- train, epoch20 -----



train_loss: 0.016216381
----------------------------------
----- val, epoch20 -----



val_loss: 0.349448711, val_acc: 0.952848723
model is saved when epoch is : 20
----------------------------------
 
learning_rate:  3.4364670044772784e-05
----- train, epoch21 -----



train_loss: 0.006189279
----------------------------------
----- val, epoch21 -----



val_loss: 0.196901426, val_acc: 0.966601179
model is saved when epoch is : 21
----------------------------------
 
